# Lab 2: Set-Covering Problem using Genetic Algorithms

In [81]:
from operator import index
import random
import sys
from time import time
import numpy as np
from collections import namedtuple
import logging
from collections import Counter


In [82]:
# PROBLEM_SIZE = 500
POPULATION_SIZE = 5
OFFSPRING_SIZE = 3

NUM_GENERATIONS = 10

N = 5

random.seed(42)

GOAL = set(range(N))

In [83]:
from traitlets.traitlets import Callable
import itertools

def remove_duplicates(list_):
  list_ = list(k for k,_ in itertools.groupby(list_))

  return list_

def sort_by(list_: list, key: Callable = None):
  list_.sort(key=key)
  return list_
  
def preproc(problem, rem_dup=False, sort=False, f:Callable = None):
  problem= sort_by(problem, key=f)
  problem= remove_duplicates(problem)
  return problem

In [84]:
Individual = namedtuple("Individual", ["genome", "fitness"])

def create_genome(problem):
    genome = []
    numbers_found = set()

    while numbers_found != GOAL:

        n_random = random.choice(range(0, len(problem)))
        genome.append(tuple(problem[n_random]))
        numbers_found |= set(problem[n_random])

        problem.pop(n_random)
        
    return genome
    
def onemax(genome):
    return sum(genome)

def fitness(genome):
    return sum(len(element) for element in genome)


def tournament(population, tournament_size=2):
    return min(random.choices(population, k=tournament_size), key=lambda i: i.fitness)


def cross_over(g1, g2):

    
    cut = random.randint(0, len(g1))
    return g1[:cut] + g2[cut:]


def mutation(g, problem):

    point = random.randint(0, len(g) - 1)
    g.pop(point)

    numbers_found = set()
    for element in g:
        numbers_found != set(element)

    steps = 0

    while numbers_found != GOAL:
        steps += 1

        if steps == 10000:
            return None
        n_random = random.choice(range(0, len(problem)))

        if not any(list == tuple(problem[n_random]) for list in g):
            g.append(tuple(problem[n_random]))
            numbers_found |= set(problem[n_random])

            problem.pop(n_random)

    return g

**Problem Definition**

In [85]:
def problem(N, seed=42):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

## Genetic Algorithm

### Initial Population

In [86]:
# population = list()

# all_list = preproc(problem(N))

# for _ in range(POPULATION_SIZE):
    
#     genome = create_genome(all_list.copy())
#     population.append(Individual(genome, fitness(genome)))

# logging.info(f"init: pop_size={len(population)}; min={min(population, key=lambda i: i.fitness)[1]}")

In [87]:
print(population)

[Individual(genome=[(0, 2), (2, 3), (2, 4), (0, 1)], fitness=8), Individual(genome=[(1, 3), (0, 1), (1,), (2, 4)], fitness=7), Individual(genome=[(1,), (0, 2), (4,), (2,), (2, 3), (2, 4), (0, 1)], fitness=7), Individual(genome=[(3,), (2, 3), (4,), (0,), (0, 2), (2, 4), (0, 1), (2,), (1, 3)], fitness=6), Individual(genome=[(1,), (0,), (2, 4), (2,), (2, 3), (3,), (0, 2), (0, 1), (4,)], fitness=7), [Individual(genome=[(1,), (0,), (2, 4), (2,), (2, 3), (3,), (0, 2), (0, 1), (4,)], fitness=13), Individual(genome=[(1,), (0, 2), (4,), (2,), (2, 3), (2, 4), (0, 1)], fitness=11), Individual(genome=[(3,), (2, 3), (4,), (0,), (0, 2), (2, 4), (0, 1), (2,), (1, 3)], fitness=14)]]


### Evolution

In [88]:
# fitness_log = [(0, i.fitness) for i in population]
population = list()

all_list = preproc(problem(N))

for _ in range(POPULATION_SIZE):
    
    genome = create_genome(all_list.copy())
    population.append(Individual(genome, fitness(genome)))

logging.info(f"init: pop_size={len(population)}; min={min(population, key=lambda i: i.fitness)[1]}")

print("aKSKAs ", population, "\n\n")
for g in range(NUM_GENERATIONS):
    offspring = list()
    print(f'population Before OFFSPRING: {population}')
    print('\n\n')
    for i in range(OFFSPRING_SIZE):
        # if random.random() < 0.3:
        p = tournament(population.copy())
        o = mutation(p.genome.copy(), all_list.copy())

        if o == None:
            continue
        # else:
        #     p1 = tournament(population)
        #     p2 = tournament(population)
        #     o = cross_over(p1.genome, p2.genome)
        f = fitness(o)
        # fitness_log.append((g + 1, f))
        print(f'0: {o}, fitness: {f}')
        offspring.append(Individual(o, fitness(o)))
        print(f'Offspingsss 0: {o}, fitness: {f}\n\n')
        
    print(f'second ppulation: {population}\n\n')
    population+=offspring
    print(f'ppulation: {population}')
    print('\n\n')
    population = sorted(population, key=lambda i: i.fitness)[:POPULATION_SIZE]
    

print("Winner ",population[0])

aKSKAs  [Individual(genome=[(0, 2), (2, 3), (2, 4), (0, 1)], fitness=8), Individual(genome=[(1, 3), (0, 1), (1,), (2, 4)], fitness=7), Individual(genome=[(1,), (0, 2), (4,), (2,), (1, 3)], fitness=7), Individual(genome=[(3,), (1,), (2, 3), (4,), (0,)], fitness=6), Individual(genome=[(1,), (0,), (2, 4), (2,), (1, 3)], fitness=7)] 


population Before OFFSPRING: [Individual(genome=[(0, 2), (2, 3), (2, 4), (0, 1)], fitness=8), Individual(genome=[(1, 3), (0, 1), (1,), (2, 4)], fitness=7), Individual(genome=[(1,), (0, 2), (4,), (2,), (1, 3)], fitness=7), Individual(genome=[(3,), (1,), (2, 3), (4,), (0,)], fitness=6), Individual(genome=[(1,), (0,), (2, 4), (2,), (1, 3)], fitness=7)]



0: [(1,), (0,), (2, 4), (2,), (2, 3), (3,), (0, 2), (0, 1), (4,)], fitness: 13
Offspingsss 0: [(1,), (0,), (2, 4), (2,), (2, 3), (3,), (0, 2), (0, 1), (4,)], fitness: 13


0: [(1,), (0, 2), (4,), (2,), (2, 3), (2, 4), (0, 1)], fitness: 11
Offspingsss 0: [(1,), (0, 2), (4,), (2,), (2, 3), (2, 4), (0, 1)], fitne

AttributeError: 'list' object has no attribute 'fitness'